In [1]:

from os.path import abspath
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *


spark=SparkSession.builder.master("local[1]")\
        .appName("Music")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mssql-jdbc-7.4.1.jre8.jar" ) \
        .config("spark.sql.warehouse.dir", '/Users/eduardoalberto/LoadFile/warehouse') \
        .config("spark.sql.catalogImplementation", "hive") \
        .enableHiveSupport() \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 

spark.sql("create database if not exists proddb150")
spark.sql("show databases;")

24/10/15 21:54:25 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.149 instead (on interface en0)
24/10/15 21:54:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/15 21:54:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[namespace: string]

In [2]:
   
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/data.csv') 
df = df.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                         "regexp_replace(trim(artists),'[^a-zA-Z0-9]',' ') as artists",
                         "cast(danceability as decimal(3,2)) as danceability ",
                         "cast(energy as decimal(3,2)) as energy",
                         "duration_ms",
                         "explicit",
                         "id",
                         "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                         "key",
                         "cast (liveness as decimal(19,3) ) as liveness",
                         "cast(loudness as decimal(19,3)) as loudness",
                         "mode",
                         "name",
                         "popularity",
                         """ case when len(trim(release_date))<> 10 then '1990-01-10' else release_date
                                    end as release_date
                         """,
                         "cast(speechiness as decimal(5,2)) as speechiness",
                         "cast(tempo as decimal(19,3)) as tempo",
                         "cast(valence as decimal(3,2)) as valence",
                         "year",
                         "current_date() as dat_ref_carga"
                         )\
            .withColumn('release_date', expr("case when release_date = 'Whipped into Shape' then '1990-01-10' else release_date end"))\
            .withColumn('danceability', expr("case when trim(regexp_replace(danceability,'[^a-zA-Z0-9]',' ')) = 'Michael Rupert' then 0 else danceability end  "))\
            .withColumn("rowid", monotonically_increasing_id()) 


df.write.mode(saveMode="overwrite").saveAsTable("proddb150.tb_music")
spark.table("proddb150.tb_music").limit(10).show()

# df.write.mode("append").jdbc("jdbc:mysql://localhost:3306/myDbUser", "tb_music",properties={"user": "root", "password": "mysql"})



# txt03 = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/artists.csv')
# txt04 = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/playlists.csv')

+------------+--------------------+------------+------+-----------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+-------------+-----+
|acousticness|             artists|danceability|energy|duration_ms|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|  tempo|valence|year|dat_ref_carga|rowid|
+------------+--------------------+------------+------+-----------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+-------------+-----+
|        1.00|   Carl Woitschach  |        0.71|  0.20|     158648|       0|6KbQ3uYMLKb5jDxLF...|            0.56| 10|   0.151| -12.428|   1|Singende Bataillo...|         0|  1990-01-10|       0.05|118.469|   0.78|1928|   2024-10-15|    0|
|        0.99|  Robert Schumann...|     

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

tracks = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/tracks.csv')

df_tracks = tracks.selectExpr("id"
                             ,"name"
                             ,"popularity"
                             ,"duration_ms"
                             ,"explicit"
                             ,"regexp_replace('artists','[^a-zA-Z0-9]',' ') as artists "
                             ,"regexp_replace('id_artists','[^a-zA-Z0-9]',' ') as id_artists"
                             ,"case when len(trim(release_date)) <> 10  then '1922-01-01' else release_date end as release_date"
                             ,"danceability"
                             ,"energy"
                             ,"key"
                             ,"loudness"
                             ,"mode"
                             ,"speechiness"
                             ,"acousticness"
                             ,"instrumentalness"
                             ,"liveness"
                             ,"valence"
                             ,"tempo"
                             ,"time_signature")\
                        .withColumn("rowid", monotonically_increasing_id()) 
df_tracks.write.mode(saveMode="overwrite").saveAsTable("proddb150.tb_tracks")
spark.table("proddb150.tb_tracks").limit(10).show()


+--------------------+--------------------+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----+
|                  id|                name|popularity|duration_ms|explicit|artists|id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|rowid|
+--------------------+--------------------+----------+-----------+--------+-------+----------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|artists|id artists|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|    0|
|021ht4sdgPcrDgSk7...|Capítulo 2.16 - B...|         0|      9820

In [5]:

genres = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/genres_v2.csv')
genres = genres.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                  "cast(energy as decimal(3,2)) as energy",
                  "key",
                  "cast(loudness as decimal(3,2)) as loudness",
                  "mode",
                  "cast(speechiness as decimal(5,2)) as speechiness",
                  "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                  "cast (liveness as decimal(3,2) ) as liveness",
                  "cast(valence as decimal(3,2)) as valence",
                  "cast(tempo as decimal(19,3)) as tempo",
                  "type",
                  "id",
                  "uri",
                  "track_href",
                  "analysis_url",
                  "duration_ms",
                  "time_signature",
                  "genre",
                  "song_name",
                  "title",
                  "current_date() as dat_ref_carga")\
        .withColumn("rowid", monotonically_increasing_id())
        
#genres.filter("title is not null").display()

genres.write.mode(saveMode="overwrite").saveAsTable("proddb150.tb_genres")
spark.table("proddb150.tb_genres").limit(10).show()

+------------+------+---+--------+----+-----------+----------------+--------+-------+-------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+-----+-------------+-----+
|acousticness|energy|key|loudness|mode|speechiness|instrumentalness|liveness|valence|  tempo|          type|                  id|                 uri|          track_href|        analysis_url|duration_ms|time_signature|    genre|           song_name|title|dat_ref_carga|rowid|
+------------+------+---+--------+----+-----------+----------------+--------+-------+-------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+-----+-------------+-----+
|        0.06|  0.81|  2|   -7.36|   1|       0.42|            0.01|    0.06|   0.39|156.985|audio_features|2Vc6NJ9PW9gD9q343...|spotify:track:2Vc...|https://api.spoti..